# Problem Description
# Features present :
* Clothing ID: Integer Categorical variable that refers to the specific piece being reviewed.
* Age: Positive Integer variable of the reviewers age.
* Title: String variable for the title of the review.
* Review Text: String variable for the review body.
* Rating: Positive Ordinal Integer variable for the product score granted by the customer from 1 Worst, to 5 Best.
* Positive Feedback Count: Positive Integer documenting the number of other customers who found this review positive.
* Division Name: Categorical name of the product high level division.
* Department Name: Categorical name of the product department name.
* Class Name: Categorical name of the product class name.

# Target :

* Recommended IND: Binary variable stating where the customer recommends the product where 1 is recommended, 0 is not recommended.

# Importing Modules

In [ ]:
import numpy as np
import pandas as pd
import nltk
import random
import os
from os import path
from PIL import Image

# Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from subprocess import check_output
from wordcloud import WordCloud, STOPWORDS

# Set Plot Theme
sns.set_palette([
    "#30a2da",
    "#fc4f30",
    "#e5ae38",
    "#6d904f",
    "#8b8b8b",
])
# Alternate # plt.style.use('fivethirtyeight')

# Pre-Processing
import string
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re
from nltk.stem import PorterStemmer

# Modeling
import statsmodels.api as sm
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from nltk.util import ngrams
from collections import Counter
from gensim.models import word2vec

# Warnings
import warnings
warnings.filterwarnings('ignore')

# NLTK

NLTK stands for Natural Language Toolkit. This toolkit is one of the most powerful NLP libraries which contains packages to make machines understand human language and reply to it with an appropriate response. Tokenization, Stemming, Lemmatization, Punctuation, Character count.

Using nltk we can determine the mood of text, NLTK’s N-grams, and gensim.models’s word2vec. It also includes statsmodels.api which offers an array of linear models.

![](https://www.guru99.com/images/1/122118_0534_NLPNaturalL3.png)

# Importing the Data

In [ ]:
# Read and Peak at Data
df = pd.read_csv("../input/womens-ecommerce-clothing-reviews/Womens Clothing E-Commerce Reviews.csv",index_col =[0])
df.head()

Checking the number of unique values for each column

In [ ]:
#Number of unique values
df.nunique()

We see that almost 3000 title data is missing. But since we will not use this feature we won't trim the data further

In [ ]:
df.describe().T.drop("count",axis=1)

# Checking age distribution and positive feedback

In [ ]:
# Continous Distributions
f, ax = plt.subplots(1,3,figsize=(16,8), sharey=False)
sns.distplot(df.Age, ax=ax[0])
ax[0].set_title("Age Distribution")
ax[0].set_ylabel("Density")
sns.distplot(df["Positive Feedback Count"], ax=ax[1])
ax[1].set_title("Positive Feedback Count Distribution")
sns.distplot(np.log10((df["Positive Feedback Count"][df["Positive Feedback Count"].notnull()]+1)), ax=ax[2])
ax[2].set_title("Positive Feedback Count Distribution\n[Log 10]")
ax[2].set_xlabel("Log Positive Feedback Count")
plt.tight_layout()
plt.show()

Most of the people are between 35 - 50 years.Trend suggest that the core market segment for this clothing brand is women between 34 and 50. With its single peak and slight right tail, the distribution of age is more or less normal.

In [ ]:
df.info()

# Imputing the Data
* Dropping Clothing ID and Title
* Checking Review Text columns
* Dropping columns which don't have any review

In [ ]:
df.drop(labels =['Clothing ID','Title'],axis = 1,inplace = True)#Dropping unwanted columns

In [ ]:
df[df['Review Text'].isnull()].shape


In [ ]:
data = df[~df['Review Text'].isnull()]  #Dropping columns which don't have any review
data.shape

# Visualising Data
* Checking Rating vs Recommended IND column

In [ ]:
import plotly.express as px

px.histogram(data, x = data['Rating'], color = data["Recommended IND"])


# Visualising Class name counts 

In [ ]:
px.histogram(data, x = data['Class Name'])

In [ ]:
px.scatter(data, x="Age", y="Positive Feedback Count", facet_row="Recommended IND", facet_col="Rating",trendline="ols",category_orders={"Rating": [1,2,3,4,5],'Recommended IND':[0,1]})

# Plotting Divisions and Age with Recommended as hue

In [ ]:
px.box(data, x="Age", y="Division Name", orientation="h",color = 'Recommended IND')


# Visualising Polarity of Review Texts

In [ ]:
err1 = data['Review Text'].str.extractall("(&amp)")
err2 = data['Review Text'].str.extractall("(\xa0)")
print('with &amp',len(err1[~err1.isna()]))
print('with (\xa0)',len(err2[~err2.isna()]))

data['Review Text'] = data['Review Text'].str.replace('(&amp)','')
data['Review Text'] = data['Review Text'].str.replace('(\xa0)','')

err1 = data['Review Text'].str.extractall("(&amp)")
print('with &amp',len(err1[~err1.isna()]))
err2 = data['Review Text'].str.extractall("(\xa0)")
print('with (\xa0)',len(err2[~err2.isna()]))

In [ ]:
!pip install TextBlob
from textblob import *

data['polarity'] = data['Review Text'].map(lambda text: TextBlob(text).sentiment.polarity)
data['polarity']

In [ ]:
px.histogram(data, x = 'polarity',color="Rating", opacity = 0.5)

In [ ]:
px.box(data, y="polarity", x="Department Name", orientation="v",color = 'Recommended IND')

In [ ]:
data['review_len'] = data['Review Text'].astype(str).apply(len)
px.histogram(data, x = 'review_len' ,color = "Recommended IND")

In [ ]:
data['token_count'] = data['Review Text'].apply(lambda x: len(str(x).split()))
px.histogram(data, x = 'token_count',color = "Recommended IND")

# Reviews with Positive Polarity 

In [ ]:
sam = data.loc[data.polarity == 1,['Review Text']].sample(3).values
for i in sam:
    print(i[0])

# Reviews with Neutral Polarity 

In [ ]:
sam = data.loc[data.polarity == 0.5,['Review Text']].sample(3).values
for i in sam:
    print(i[0])

# Reviews with negative Polarity

In [ ]:
sam = data.loc[data.polarity < 0,['Review Text']].sample(3).values
for i in sam:
    print(i[0])

# Visualising the polarity of Reviews using a pie chart

In [ ]:
negative = (len(data.loc[data.polarity <0,['Review Text']].values)/len(data))*100
positive = (len(data.loc[data.polarity >0.5,['Review Text']].values)/len(data))*100
neutral  = len(data.loc[data.polarity >0 ,['Review Text']].values) - len(data.loc[data.polarity >0.5 ,['Review Text']].values)
neutral = neutral/len(data)*100

from matplotlib import pyplot as plt 
plt.figure(figsize =(10, 7)) 
plt.pie([positive,negative,neutral], labels = ['Positive','Negative','Neutral'] , colors = ["green" ,"red" ,"mediumslateblue"])

# Creating N grams 

N-grams of texts are extensively used in *text mining* and *natural language processing* tasks. They are basically a set of co-occuring words within a given window and when computing the n-grams you typically move one word forward (although you can move X words forward in more advanced scenarios). 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
def top_n_ngram(corpus,n = None,ngram = 1):
    vec = CountVectorizer(stop_words = 'english',ngram_range=(ngram,ngram)).fit(corpus)
    bag_of_words = vec.transform(corpus) #Have the count of  all the words for each review
    sum_words = bag_of_words.sum(axis =0) #Calculates the count of all the word in the whole review
    words_freq = [(word,sum_words[0,idx]) for word,idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq,key = lambda x:x[1],reverse = True)
    return words_freq[:n]

   # Visualizing Top 10 Unigrams

In [ ]:
common_words = top_n_ngram(data['Review Text'], 10,1)
df = pd.DataFrame(common_words, columns = ['ReviewText' , 'count'])
plt.figure(figsize =(10,5))
df.groupby('ReviewText').sum()['count'].sort_values(ascending=False).plot(
kind='bar', title='Top 10 unigrams in review after removing stop words')

# Visualizing Top 20 Bigrams

In [ ]:
common_words = top_n_ngram(data['Review Text'], 20,2)
df = pd.DataFrame(common_words, columns = ['ReviewText' , 'count'])
plt.figure(figsize =(10,5))
df.groupby('ReviewText').sum()['count'].sort_values(ascending=False).plot(
kind='bar', title='Top 20 bigrams in review after removing stop words')

# Visualizing Top 20 Part-of-Speech


In [ ]:
blob= TextBlob(str(data['Review Text']))
pos = pd.DataFrame(blob.tags,columns =['word','pos'])
pos1 = pos.pos.value_counts()[:20]
plt.figure(figsize = (10,5))
pos1.plot(kind='bar',title ='Top 20 Part-of-speech taggings')

# Defining our data

In [ ]:
y = data['Recommended IND']
X = data.drop(columns = 'Recommended IND')

# Visualising Heatmaps

In [ ]:
import seaborn as sns
sns.heatmap(X.corr(),annot =True, cmap = "icefire")

# Handling Multi - Colinearity

In [ ]:
set1 =set()
cor = X.corr()
for i in cor.columns:
    for j in cor.columns:
        if cor[i][j]>0.8 and i!=j:
            set1.add(i)
print(set1)
X = X.drop(labels = ['token_count'],axis = 1)
print("correlation: ", X.corr())

# Statistical Description of Data

In [ ]:
class1 =[]
for i in X.polarity:
    if float(i)>=0.0:
        class1.append(1)
    elif float(i)<0.0:
        class1.append(0)
X['sentiment'] = class1

X.groupby(X['sentiment']).describe().T

In [ ]:
print("Shape of X: " , X.shape)
print("Shape of y: " , y.shape)

# MODEL I

# Creating the model

* Stemming is the process of producing morphological variants of a root/base word. Stemming programs are commonly referred to as stemming algorithms or stemmers using PorterStemmer from NLTK


In [ ]:
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer


corpus =[]
X.index = np.arange(len(X))

In [ ]:
len(X)

# RE -> Tokenizing -> Stemming -> Corpus Creation 

In [ ]:
from tqdm import tqdm
for i in tqdm(range(len(X))):
    review = re.sub('[^a-zA-z]',' ',X['Review Text'][i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    review =[ps.stem(i) for i in review if not i in set(stopwords.words('english'))]
    review =' '.join(review)
    corpus.append(review)
corpus[0:5]

# Bag of words
To create the bag of words model, we need to create a matrix where the columns correspond to the most frequent words in our dictionary where rows correspond to the document or sentences.

![](https://i.pinimg.com/originals/dd/80/9c/dd809cd103e3f94c252b6073c474bcac.png)

In [ ]:
len(corpus)

# Creating Count Vector

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer as CV
cv  = CV(max_features = 3000,ngram_range=(1,1))
X_cv = cv.fit_transform(corpus).toarray()
y = y.values
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_cv, y, test_size = 0.20, random_state = 0)
from sklearn.naive_bayes import BernoulliNB
classifier = BernoulliNB()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)
from sklearn.metrics import accuracy_score
from sklearn import metrics
acc = accuracy_score(y_test, y_pred)
print("Accuracy of the classifier: ",acc)
print("Confusion matrix is :\n",metrics.confusion_matrix(y_test,y_pred))
print("Classification report: \n" ,metrics.classification_report(y_test,y_pred))

# Creating Term frequency- Inverse Document Frequency Technique (tf-idf)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer as TV
tv  = TV(ngram_range =(1,1),max_features = 3000)
X_tv = tv.fit_transform(corpus).toarray()
X_train, X_test, y_train, y_test = train_test_split(X_tv, y, test_size = 0.20, random_state = 0)
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Accuracy of the classifier: ",acc)
print("Confusion matrix is :\n",metrics.confusion_matrix(y_test,y_pred))
print("Classification report: \n" ,metrics.classification_report(y_test,y_pred))

# Creating a ANN structure

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(num_words = 3000)
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)
padded = pad_sequences(sequences, padding='post')
word_index = tokenizer.word_index
count = 0
for i,j in word_index.items():
    if count == 11:
        break
    print(i,j)
    count = count+1

In [ ]:
embedding_dim = 64
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(3000, embedding_dim),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

In [ ]:
num_epochs = 10

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(padded,y,epochs= num_epochs,validation_split= 0.39)

# Visualising Model history

In [ ]:
loss = model.history.history
loss = pd.DataFrame(loss)

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
t = f.suptitle('Basic ANN Performance', fontsize=12)
f.subplots_adjust(top=0.85, wspace=0.3)

epoch_list = range(1,11)
ax1.plot(epoch_list, loss['accuracy'], label='Train Accuracy')
ax1.plot(epoch_list, loss['val_accuracy'], label='Validation Accuracy')
ax1.set_xticks(np.arange(0, 11, 1))
ax1.set_ylabel('Accuracy Value')
ax1.set_xlabel('Epoch')
ax1.set_title('Accuracy')
l1 = ax1.legend(loc="best")

ax2.plot(epoch_list, loss['loss'], label='Train Loss')
ax2.plot(epoch_list, loss['val_loss'], label='Validation Loss')
ax2.set_xticks(np.arange(0, 11, 1))
ax2.set_ylabel('Loss Value')
ax2.set_xlabel('Epoch')
ax2.set_title('Loss')
l2 = ax2.legend(loc="best")

# Checking the model on a random example
* Check if the review will be recommended or not.

In [ ]:
sample_string = "I hate it"
sample = tokenizer.texts_to_sequences(sample_string)
padded_sample = pad_sequences(sample, padding='post')
print("Padded sample", padded_sample.T)
print("Probabilty of a person recommending :",model.predict(padded_sample.T)[0][0]*100,"%")

Hence, the model predicts that for a comment "I hate it" there is only 24.61% chance of recommending it to someone

In [ ]:
sample_string = "i love the fabric"
sample = tokenizer.texts_to_sequences(sample_string)
padded_sample = pad_sequences(sample, padding='post')
print("Padded sample", padded_sample.T)
print("Probabilty of a person recommending :",model.predict(padded_sample.T)[0][0]*100,"%")

Hence, the model predicts that for a comment "I love the fabric" there is 99.98 % chance of recommending it to someone

# Model II

# Creating a model to check Recommended

* Creating X and y for training and test data

In [ ]:
y = data['Recommended IND'].tolist()
X = list(data["Review Text"])

# Separate out the sentences and labels into training and test sets
training_size = int(len(X) * 0.8)

training_sentences = X[0:training_size]
testing_sentences = X[training_size:]
training_labels = y[0:training_size]
testing_labels = y[training_size:]

# Make labels into numpy arrays for use with the network later
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

# Creating Corpus

In [ ]:
vocab_size = 1000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, padding=padding_type, 
                       truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length, 
                               padding=padding_type, truncating=trunc_type)

In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[1]))
print(training_sentences[1])

In [ ]:
# Build a basic sentiment network
# Note the embedding layer is first, 
# and the output is only 1 node as it is either 0 or 1 (negative or positive)
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

In [ ]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

In [ ]:
loss = model.history.history
loss = pd.DataFrame(loss)
loss

In [ ]:


f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
t = f.suptitle('Basic ANN Performance', fontsize=12)
f.subplots_adjust(top=0.85, wspace=0.3)

epoch_list = range(1,11)
ax1.plot(epoch_list, loss['accuracy'], label='Train Accuracy')
ax1.plot(epoch_list, loss['val_accuracy'], label='Validation Accuracy')
ax1.set_xticks(np.arange(0, 11, 1))
ax1.set_ylabel('Accuracy Value')
ax1.set_xlabel('Epoch')
ax1.set_title('Accuracy')
l1 = ax1.legend(loc="best")

ax2.plot(epoch_list, loss['loss'], label='Train Loss')
ax2.plot(epoch_list, loss['val_loss'], label='Validation Loss')
ax2.set_xticks(np.arange(0, 11, 1))
ax2.set_ylabel('Loss Value')
ax2.set_xlabel('Epoch')
ax2.set_title('Loss')
l2 = ax2.legend(loc="best")